### 1) Tracking and saving without showing the video

In [ ]:
#### Code to find coordinates not showing video

import cv2
import sys 
import math
import csv
import pandas as pd
import os
from shapely.geometry import Polygon, Point
import numpy as np
import shutil

sys.path.append("eel_detection")
import object_detection
from object_detection import ObjectDetection

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

video_list_path = list(clear_filepath("eel_detection/videos/to_analyse"))
print(video_list_path)

od= ObjectDetection()

def dist(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

past1 = {}
past2 = {}
past3 = {}
past4 = {}
past5 = {}
past6 = {}
past7 = {}
past8 = {}
past9 = {}
past10 = {}
past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]


file = open("create_boundaries/Box_boundaries.csv","r")

data=[]
for n in file:
    n=n.strip("\n")
    n = n.replace('"', "")
    data.append(n)   

bounds=[]
for n in data:
    if n == "video_name,new_col_x,new_col_y":
        continue
    else:
        n= n.split(",")
        bounds.append(n)    


for video in video_list_path :
    
    file_name = os.path.basename(video)
    vid_name = file_name.removesuffix(".mp4")
    
    df = pd.DataFrame(columns=["Video", "Frame", "Eel", "Coordinates"])
    
    df_check = pd.DataFrame(columns=["Video", "Frame", "Box"])
    
    print(video)
    cap = cv2.VideoCapture(video)
    
    current_vid=[]
    for n in bounds:
        for item in n:
            if file_name.removesuffix(".mp4") == item:
                n.pop(0)
                current_vid.append(n)
    
    if len(current_vid) == 0:
        dest_miss = ("eel_detection/videos/missing_boundaries/" + os.path.basename(video))
        cap.release()
        shutil.move(video, dest_miss)
        continue
    
    test=[]
    for n in current_vid:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    real_bound_tl=test[0:4]
    real_bound_tr=test[4:8]
    real_bound_br=test[8:12]
    real_bound_bl=test[12:16]
    
    polygon_tl=Polygon(real_bound_tl)
    polygon_tr=Polygon(real_bound_tr)
    polygon_br=Polygon(real_bound_br)
    polygon_bl=Polygon(real_bound_bl)
    
    polygon_dict = {1: (polygon_tl), 2: (polygon_tr), 3: (polygon_bl), 4: (polygon_br)}
    
    used_IDs = []
    
    count = 0
    track_id=5
    center_points_prev=[]
    tracking_objects={}
    
    while True:
        
        ret, frame= cap.read()
        count = count + 1
        if not ret:
            break
        
        center_points_current=[]
        
        (class_ids, scores, boxes) =od.detect(frame)
        
        tracking_objects_boxes={}
        
        for box in boxes:
            (x,y,w,h)=box
            
            cx= x+w/2
            cy= y+h/2
            
            cy_new = (cap.get(4)-(y+h/2))
            
            center_points_current.append((cx,cy_new))
        
        if count <=2:
            for pt in center_points_current:
                contained = False
                for pt2 in center_points_prev:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        for pos, polygon in polygon_dict.items():
                            if contained == True:
                                break
                            if polygon.contains(Point(pt)) == True:
                                contained = True
                                if pos in tracking_objects_boxes.keys():
                                    if pos == 1:
                                        box = "top-left"
                                    if pos == 2:
                                        box = "top-right"
                                    if pos == 3:
                                        box = "bottom-left"
                                    if pos == 4:
                                        box = "bottom-right"
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                                 "Box": [box]})], ignore_index=True)
                                    tracking_objects[track_id] = pt
                                    used_IDs.append(track_id)
                                    track_id = track_id + 1
                                else:
                                    tracking_objects[pos] = pt
                                    used_IDs.append(pos)
                                    tracking_objects_boxes[pos] = pt
                            
        else:
            tracking_objects_copy = tracking_objects.copy()
            center_points_current_copy = center_points_current.copy()
            
            for object_id, pt2 in tracking_objects_copy.items():
                object_exists = False
                for pt in center_points_current_copy:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[object_id]= pt
                        object_exists = True
                        for pos, polygon in polygon_dict.items():
                                if polygon.contains(Point(pt)) == True:
                                    if pos in tracking_objects_boxes.keys():
                                        if pos == 1:
                                            box = "top-left"
                                        if pos == 2:
                                            box = "top-right"
                                        if pos == 3:
                                            box = "bottom-left"
                                        if pos == 4:
                                            box = "bottom-right"
                                        df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                                     "Box": [box]})], ignore_index=True)
                                    else:
                                        tracking_objects_boxes[pos]=pt
                        if pt in center_points_current:
                            center_points_current.remove(pt)
                        continue
                
                if not object_exists:
                    past1[object_id] = tracking_objects[object_id]
                    tracking_objects.pop(object_id)
            
            for pt in center_points_current:
                found = False
                for past in past_list:
                    past_copy = past.copy()
                    if found == True:
                        break
                    
                    for object_id, pt2 in past_copy.items():
                        distance = dist(pt, pt2)
                        
                        if distance < 50:
                            tracking_objects.update({object_id: pt})
                            found = True
                            past.pop(object_id)
                            continue
                        
                        if found == True:
                            break
                
                if found == False:
                    for pos, polygon in polygon_dict.items():
                        if polygon.contains(Point(pt)) == True:
                            if pos in tracking_objects_boxes.keys():
                                tracking_objects[track_id] = pt
                                used_IDs.append(track_id)
                                track_id = track_id + 1
                            else:
                                in_past = False
                                for used in used_IDs:
                                    if in_past == True:
                                        break
                                    if used == pos:
                                        in_past = True
                                        tracking_objects[track_id] = pt
                                        used_IDs.append(track_id)
                                        track_id = track_id + 1
                                if in_past == False:
                                    tracking_objects[pos] = pt
            
                for pos, polygon in polygon_dict.items():
                    if polygon.contains(Point(pt)) == True:
                        if pos in tracking_objects_boxes.keys():
                            if pos == 1:
                                box = "top-left"
                            if pos == 2:
                                box = "top-right"
                            if pos == 3:
                                box = "bottom-left"
                            if pos == 4:
                                box = "bottom-right"
                            df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                          "Box": [box]})], ignore_index=True)
                        else:
                            tracking_objects_boxes[pos]=pt
                    

        past10 = past9
        past9 = past8
        past8 = past7
        past7 = past6
        past6 = past5
        past5 = past4
        past4 = past3
        past3 = past2
        past2 = past1
        past1 = {}
        past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]
        
        print("Frame", count, ": " ,tracking_objects)
        
        df_dictionary = pd.DataFrame(tracking_objects.items(), columns = ["Eel", "Coordinates"])
        df_dictionary["Frame"] = count
        df_dictionary["Video"] = vid_name
        df = pd.concat([df, df_dictionary], ignore_index=True)
        
        center_points_prev= center_points_current.copy()
        
        key= cv2.waitKey(1)
        if key == 27:
            break
    
    cap.release()
    
    df.to_csv("eel_detection/videos/Eel_coordinates.csv", index=False, mode="a")
    dest_done = ("eel_detection/videos/analysed/" + os.path.basename(video))
    shutil.move(video, dest_done)
    
    if len(df_check) > 0:
        df_check.to_csv("eel_detection/videos/Dupes_in_boxes.csv", index=False, mode="a")


print("Done")

### 2) Tracking and saving showing video

In [ ]:
#### Code to find coordinates showing video

import cv2
import sys 
import math
import csv
import pandas as pd
import os
from shapely.geometry import Polygon, Point
import numpy as np
import shutil

sys.path.append("eel_detection")
import object_detection
from object_detection import ObjectDetection

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

video_list_path = list(clear_filepath("eel_detection/videos/to_analyse"))
print(video_list_path)

od= ObjectDetection()

def dist(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

past1 = {}
past2 = {}
past3 = {}
past4 = {}
past5 = {}
past6 = {}
past7 = {}
past8 = {}
past9 = {}
past10 = {}
past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]


file = open("create_boundaries/Box_boundaries.csv","r")

data=[]
for n in file:
    n=n.strip("\n")
    n = n.replace('"', "")
    data.append(n)   

bounds=[]
for n in data:
    if n == "video_name,new_col_x,new_col_y":
        continue
    else:
        n= n.split(",")
        bounds.append(n)    


for video in video_list_path :
    
    file_name = os.path.basename(video)
    vid_name = file_name.removesuffix(".mp4")
    
    df = pd.DataFrame(columns=["Video", "Frame", "Eel", "Coordinates"])
    
    
    df_check = pd.DataFrame(columns=["Video", "Frame", "Box"])
    
    print(video)
    cap = cv2.VideoCapture(video)
    
    current_vid=[]
    for n in bounds:
        for item in n:
            if file_name.removesuffix(".mp4") == item:
                n.pop(0)
                current_vid.append(n)
    
    if len(current_vid) == 0:
        dest_miss = ("eel_detection/videos/missing_boundaries/" + os.path.basename(video))
        cap.release()
        shutil.move(video, dest_miss)
        continue
    
    test=[]
    for n in current_vid:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    test_vid=[]
    for n in current_vid:
        n=(round(float(n[0])), round(cap.get(4)-float(n[1])))
        test_vid.append(n)
    
    real_bound_tl=test[0:4]
    real_bound_tr=test[4:8]
    real_bound_br=test[8:12]
    real_bound_bl=test[12:16]
    
    polygon_tl=Polygon(real_bound_tl)
    polygon_tr=Polygon(real_bound_tr)
    polygon_br=Polygon(real_bound_br)
    polygon_bl=Polygon(real_bound_bl)
    
    polygon_dict = {1: (polygon_tl), 2: (polygon_tr), 3: (polygon_bl), 4: (polygon_br)}
    
    vid_bound_tl=test_vid[0:4]
    vid_bound_tr=test_vid[4:8]
    vid_bound_br=test_vid[8:12]
    vid_bound_bl=test_vid[12:16]
    
    tl_array = np.array([vid_bound_tl], np.int32)
    tr_array = np.array([vid_bound_tr], np.int32)
    br_array = np.array([vid_bound_br], np.int32)
    bl_array = np.array([vid_bound_bl], np.int32)
    
    array_list = (tl_array, tr_array, br_array, bl_array)
    
    used_IDs = []
    
    count = 0
    track_id=5
    center_points_prev=[]
    tracking_objects={}
    
    while True:
        
        ret, frame= cap.read()
        count = count + 1
        if not ret:
            break
        
        center_points_current=[]
        
        (class_ids, scores, boxes) =od.detect(frame)
        
        tracking_objects_boxes={}
        
        for box in boxes:
            (x,y,w,h)=box
            
            cx= x+w/2
            cy= y+h/2
            cv2.circle(frame,(int(cx),int(cy)),5,(0,0,120),-1)  
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,120),2)
            
            cy_new = (cap.get(4)-(y+h/2))
            
            center_points_current.append((cx,cy_new))

        
        for container in array_list:
            cv2.polylines(frame, [container], True, (0,120,0))
        
        if count <=2:
            for pt in center_points_current:
                contained = False
                for pt2 in center_points_prev:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        for pos, polygon in polygon_dict.items():
                            if contained == True:
                                break
                            if polygon.contains(Point(pt)) == True:
                                contained = True
                                if pos in tracking_objects_boxes.keys():
                                    if pos == 1:
                                        box = "top-left"
                                    if pos == 2:
                                        box = "top-right"
                                    if pos == 3:
                                        box = "bottom-left"
                                    if pos == 4:
                                        box = "bottom-right"
                                    df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                                 "Box": [box]})], ignore_index=True)
                                    tracking_objects[track_id] = pt
                                    used_IDs.append(track_id)
                                    track_id = track_id + 1
                                else:
                                    tracking_objects[pos] = pt
                                    used_IDs.append(pos)
                                    tracking_objects_boxes[pos] = pt
                            
        else:
            tracking_objects_copy = tracking_objects.copy()
            center_points_current_copy = center_points_current.copy()
            
            for object_id, pt2 in tracking_objects_copy.items():
                object_exists = False
                for pt in center_points_current_copy:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[object_id]= pt
                        object_exists = True
                        for pos, polygon in polygon_dict.items():
                                if polygon.contains(Point(pt)) == True:
                                    if pos in tracking_objects_boxes.keys():
                                        if pos == 1:
                                            box = "top-left"
                                        if pos == 2:
                                            box = "top-right"
                                        if pos == 3:
                                            box = "bottom-left"
                                        if pos == 4:
                                            box = "bottom-right"
                                        df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                                     "Box": [box]})], ignore_index=True)
                                    else:
                                        tracking_objects_boxes[pos]=pt
                        if pt in center_points_current:
                            center_points_current.remove(pt)
                        continue
                
                if not object_exists:
                    past1[object_id] = tracking_objects[object_id]
                    tracking_objects.pop(object_id)
            
            for pt in center_points_current:
                found = False
                for past in past_list:
                    past_copy = past.copy()
                    if found == True:
                        break
                    
                    for object_id, pt2 in past_copy.items():
                        distance = dist(pt, pt2)
                        
                        if distance < 50:
                            tracking_objects.update({object_id: pt})
                            found = True
                            past.pop(object_id)
                            continue
                        
                        if found == True:
                            break
                
                if found == False:
                    for pos, polygon in polygon_dict.items():
                        if polygon.contains(Point(pt)) == True:
                            if pos in tracking_objects_boxes.keys():
                                tracking_objects[track_id] = pt
                                used_IDs.append(track_id)
                                track_id = track_id + 1
                            else:
                                in_past = False
                                for used in used_IDs:
                                    if in_past == True:
                                        break
                                    if used == pos:
                                        in_past = True
                                        tracking_objects[track_id] = pt
                                        used_IDs.append(track_id)
                                        track_id = track_id + 1
                                if in_past == False:
                                    tracking_objects[pos] = pt
            
                for pos, polygon in polygon_dict.items():
                    if polygon.contains(Point(pt)) == True:
                        if pos in tracking_objects_boxes.keys():
                            if pos == 1:
                                box = "top-left"
                            if pos == 2:
                                box = "top-right"
                            if pos == 3:
                                box = "bottom-left"
                            if pos == 4:
                                box = "bottom-right"
                            df_check = pd.concat([df_check, pd.DataFrame({"Video": [vid_name], "Frame": [count], 
                                                                          "Box": [box]})], ignore_index=True)
                        else:
                            tracking_objects_boxes[pos]=pt
                    

        past10 = past9
        past9 = past8
        past8 = past7
        past7 = past6
        past6 = past5
        past5 = past4
        past4 = past3
        past3 = past2
        past2 = past1
        past1 = {}
        past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]
        
        for object_id, pt in tracking_objects.items():
            cv2.circle(frame,(int(pt[0]), int(cap.get(4)-pt[1])),5,(0,0,120),-1)
            cv2.putText(frame,str(object_id),(int(pt[0]),int(cap.get(4)-pt[1])-7),0,1,(0,0,120),0)
        
        print("Frame", count, ": " ,tracking_objects)
        
        df_dictionary = pd.DataFrame(tracking_objects.items(), columns = ["Eel", "Coordinates"])
        df_dictionary["Frame"] = count
        df_dictionary["Video"] = vid_name
        df = pd.concat([df, df_dictionary], ignore_index=True)
        
        cv2.imshow("eel",frame)
        
        center_points_prev= center_points_current.copy()
        
        key= cv2.waitKey(1)
        if key == 27:
            break
    
    cap.release()
    
    df.to_csv("eel_detection/videos/Eel_coordinates.csv", index=False, mode="a")
    dest_done = ("eel_detection/videos/analysed/" + os.path.basename(video))
    shutil.move(video, dest_done)
    
    if len(df_check) > 0:
        df_check.to_csv("eel_detection/videos/Dupes_in_boxes.csv", index=False, mode="a")
        
    cv2.destroyAllWindows()
    
print("Done")

### 3) Tracking with video without saving

In [ ]:
#### Code to show video without saving coordinates

import cv2
import sys 
import math
import csv
import pandas as pd
import os
from shapely.geometry import Polygon, Point
import numpy as np
import shutil

sys.path.append("eel_detection")
import object_detection
from object_detection import ObjectDetection

def clear_filepath(directory):
    for dirpath,_, filenames in os.walk(directory):
        for files in filenames:
            if not files.startswith("."):
                yield os.path.abspath(os.path.join(dirpath,files))

video_list_path = list(clear_filepath("eel_detection/videos/to_analyse"))
print(video_list_path)

od= ObjectDetection()

def dist(pt1, pt2):
    return(math.hypot(pt2[0]-pt1[0],pt2[1]-pt1[1]))

past1 = {}
past2 = {}
past3 = {}
past4 = {}
past5 = {}
past6 = {}
past7 = {}
past8 = {}
past9 = {}
past10 = {}
past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]


file = open("create_boundaries/Box_boundaries.csv","r")

data=[]
for n in file:
    n=n.strip("\n")
    n = n.replace('"', "")
    data.append(n)   

bounds=[]
for n in data:
    if n == "video_name,new_col_x,new_col_y":
        continue
    else:
        n= n.split(",")
        bounds.append(n)    


for video in video_list_path :
    
    file_name = os.path.basename(video)
    
    print(video)
    cap = cv2.VideoCapture(video)
    
    current_vid=[]
    for n in bounds:
        for item in n:
            if file_name.removesuffix(".mp4") == item:
                n.pop(0)
                current_vid.append(n)
    
    if len(current_vid) == 0:
        dest_miss = ("eel_detection/videos/missing_boundaries/" + os.path.basename(video))
        cap.release()
        shutil.move(video, dest_miss)
        continue
    
    test=[]
    for n in current_vid:
        n=(float(n[0]),float(n[1]))
        test.append(n)
    
    test_vid=[]
    for n in current_vid:
        n=(round(float(n[0])), round(cap.get(4)-float(n[1])))
        test_vid.append(n)
    
    real_bound_tl=test[0:4]
    real_bound_tr=test[4:8]
    real_bound_br=test[8:12]
    real_bound_bl=test[12:16]
    
    polygon_tl=Polygon(real_bound_tl)
    polygon_tr=Polygon(real_bound_tr)
    polygon_br=Polygon(real_bound_br)
    polygon_bl=Polygon(real_bound_bl)
    
    polygon_dict = {1: (polygon_tl), 2: (polygon_tr), 3: (polygon_bl), 4: (polygon_br)}
    
    vid_bound_tl=test_vid[0:4]
    vid_bound_tr=test_vid[4:8]
    vid_bound_br=test_vid[8:12]
    vid_bound_bl=test_vid[12:16]
    
    tl_array = np.array([vid_bound_tl], np.int32)
    tr_array = np.array([vid_bound_tr], np.int32)
    br_array = np.array([vid_bound_br], np.int32)
    bl_array = np.array([vid_bound_bl], np.int32)
    
    array_list = (tl_array, tr_array, br_array, bl_array)
    
    used_IDs = []
    
    count = 0
    track_id=5
    center_points_prev=[]
    tracking_objects={}
    
    while True:
        
        ret, frame= cap.read()
        count = count + 1
        if not ret:
            break
        
        center_points_current=[]
        
        (class_ids, scores, boxes) =od.detect(frame)
        
        tracking_objects_boxes={}
        
        for box in boxes:
            (x,y,w,h)=box
            
            cx= x+w/2
            cy= y+h/2
            cv2.circle(frame,(int(cx),int(cy)),5,(0,0,120),-1)  
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,120),2)
            
            cy_new = (cap.get(4)-(y+h/2))
            
            center_points_current.append((cx,cy_new))

        
        for container in array_list:
            cv2.polylines(frame, [container], True, (0,120,0))
        
        if count <=2:
            for pt in center_points_current:
                contained = False
                for pt2 in center_points_prev:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        for pos, polygon in polygon_dict.items():
                            if contained == True:
                                break
                            if polygon.contains(Point(pt)) == True:
                                contained = True
                                if pos in tracking_objects_boxes.keys():
                                    if pos == 1:
                                        box = "top-left"
                                    if pos == 2:
                                        box = "top-right"
                                    if pos == 3:
                                        box = "bottom-left"
                                    if pos == 4:
                                        box = "bottom-right"
                                    tracking_objects[track_id] = pt
                                    used_IDs.append(track_id)
                                    track_id = track_id + 1
                                else:
                                    tracking_objects[pos] = pt
                                    used_IDs.append(pos)
                                    tracking_objects_boxes[pos] = pt
                            
        else:
            tracking_objects_copy = tracking_objects.copy()
            center_points_current_copy = center_points_current.copy()
            
            for object_id, pt2 in tracking_objects_copy.items():
                object_exists = False
                for pt in center_points_current_copy:
                    distance= dist(pt, pt2)
                    if distance < 50:
                        tracking_objects[object_id]= pt
                        object_exists = True
                        for pos, polygon in polygon_dict.items():
                                if polygon.contains(Point(pt)) == True:
                                    if pos in tracking_objects_boxes.keys():
                                        if pos == 1:
                                            box = "top-left"
                                        if pos == 2:
                                            box = "top-right"
                                        if pos == 3:
                                            box = "bottom-left"
                                        if pos == 4:
                                            box = "bottom-right"
                                    else:
                                        tracking_objects_boxes[pos]=pt
                        if pt in center_points_current:
                            center_points_current.remove(pt)
                        continue
                
                if not object_exists:
                    past1[object_id] = tracking_objects[object_id]
                    tracking_objects.pop(object_id)
            
            for pt in center_points_current:
                found = False
                for past in past_list:
                    past_copy = past.copy()
                    if found == True:
                        break
                    
                    for object_id, pt2 in past_copy.items():
                        distance = dist(pt, pt2)
                        
                        if distance < 50:
                            tracking_objects.update({object_id: pt})
                            found = True
                            past.pop(object_id)
                            continue
                        
                        if found == True:
                            break
                
                if found == False:
                    for pos, polygon in polygon_dict.items():
                        if polygon.contains(Point(pt)) == True:
                            if pos in tracking_objects_boxes.keys():
                                tracking_objects[track_id] = pt
                                used_IDs.append(track_id)
                                track_id = track_id + 1
                            else:
                                in_past = False
                                for used in used_IDs:
                                    if in_past == True:
                                        break
                                    if used == pos:
                                        in_past = True
                                        tracking_objects[track_id] = pt
                                        used_IDs.append(track_id)
                                        track_id = track_id + 1
                                if in_past == False:
                                    tracking_objects[pos] = pt
            
                for pos, polygon in polygon_dict.items():
                    if polygon.contains(Point(pt)) == True:
                        if pos in tracking_objects_boxes.keys():
                            if pos == 1:
                                box = "top-left"
                            if pos == 2:
                                box = "top-right"
                            if pos == 3:
                                box = "bottom-left"
                            if pos == 4:
                                box = "bottom-right"
                        else:
                            tracking_objects_boxes[pos]=pt
                    

        past10 = past9
        past9 = past8
        past8 = past7
        past7 = past6
        past6 = past5
        past5 = past4
        past4 = past3
        past3 = past2
        past2 = past1
        past1 = {}
        past_list = [past1, past2, past3, past4, past5, past6, past7, past8, past9, past10]
        
        for object_id, pt in tracking_objects.items():
            cv2.circle(frame,(int(pt[0]), int(cap.get(4)-pt[1])),5,(0,0,120),-1)
            cv2.putText(frame,str(object_id),(int(pt[0]),int(cap.get(4)-pt[1])-7),0,1,(0,0,120),0)
        
        print(count,"track",tracking_objects)
        
        cv2.imshow("eel",frame)
        
        center_points_prev= center_points_current.copy()
        
        key= cv2.waitKey(1)
        if key == 27:
            break
    
    cap.release()
    
    dest_done = ("eel_detection/videos/analysed/" + os.path.basename(video))
    shutil.move(video, dest_done)
        
    cv2.destroyAllWindows()
    
print("done")